## 맛집 주소 크롤링 , 위/경도는 카카오 API

### 샘플링한 맛집 자료 주소 크롤링하기

In [2]:
import pandas as pd
import numpy as np
import time
from time import sleep
import json
from urllib.parse import quote
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [6]:
df = pd.read_csv('con_data/맛집_sample.csv')
df

,CMPNM_NM,ADSTRD_NM,MLSFC_NM,SCLAS_NM,JJINHBT_SALES_PRICE_RATE,JJINHBT_SALES_CO_RATE,OTSD_SALES_PRICE_RATE,OTSD_SALES_CO_RATE,ALL_SALES_PRICE_RATE,ALL_SALES_CO_RATE
0,청춘두루치기,노형동,한식,가정식,0.392500,0.362500,0.002500,0.012500,0.052500,0.135000
1,원풍해장국,조천읍,한식,가정식,0.665000,0.841250,0.020000,0.151250,0.060000,0.425000
2,돼지,일도2동,아시아음식,중식,0.110000,0.200000,0.000000,0.000000,0.010000,0.110000
3,홍리짬뽕,동홍동,한식,중식,0.301818,0.540000,0.004545,0.040000,0.035455,0.240909
4,돌담집,오라동,한식,단품요리 전문,0.300000,0.295000,0.025000,0.035000,0.065000,0.135000
...,...,...,...,...,...,...,...,...,...,...
1995,삼성웰스토리 (주)호텔신라제주면세점,연동,한식,구내식당/푸드코트,0.340000,0.365000,0.000000,0.002500,0.025000,0.152500
1996,제주돈애월점,애월읍,한식,돼지고기,0.697273,0.259091,0.150000,0.206364,0.217273,0.268182
1997,가을남자의커피향기,애월읍,한식,단품요리 전문,0.090000,0.252500,0.007500,0.147500,0.013750,0.216250
1998,1970새마을포차 제주시청점,이도2동,주점및주류판매,포장마차,0.958571,0.652143,0.028571,0.120000,0.085714,0.297857


In [7]:
options = webdriver.ChromeOptions()
 
# 창 숨기는 옵션 추가
options.add_argument("headless")

In [8]:
data=[]
data_n =[]

for place in df['CMPNM_NM']:
        kakao_map_search_url = f"https://map.kakao.com/?q={place}"
        driver = webdriver.Chrome(options=options) #크롬창 띄우기
        driver.get(kakao_map_search_url) #위의 url로 이동
        driver.implicitly_wait(time_to_wait=3) #url로 이동하기까지 3초 기다려줌
        soup = BeautifulSoup(driver.page_source,'html.parser') 
        #불러온 html페이지를 뷰티풀솦으로 Pasing함       
        try:
                address = driver.find_element(By.XPATH, '//*[@id="info.search.place.list"]/li/div[5]/div[2]/p[1]').text
                tel = driver.find_element(By.XPATH,'//*[@id="info.search.place.list"]/li/div[5]/div[4]/span[1]').text
                data.append({'이름': place, '주소':address,'전화번호': tel})
                time.sleep(5)
        except:
                data_n.append({'이름': place})

In [ ]:
df1 = pd.DataFrame(data)
df2 = pd.DataFrame(data_n)

In [ ]:
sample = df1[(df1['주소'].apply(lambda x: x.find('제주')) == 0)].reset_index(drop=True)
sample #주소에 제주가 들어가는 데이터만 뽑기

### 카카오 API로 위도 경도 받아오기

In [ ]:
with open('C:\workspace\KakaoAPI.txt') as file:
    road_key = file.read()

In [ ]:
base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
lat_list, lng_list = [], []
header = {'Authorization': f'KakaoAK {road_key}'}
for i in sample.index:
    try:
        url = f'{base_url}?query={quote(sample["주소"][i])}'
        result = requests.get(url, headers=header).json()
        lat_list.append(result['documents'][0]['y'])
        lng_list.append(result['documents'][0]['x'])
    except:
        print(sample.이름[i])

In [ ]:
sample[sample['이름']=='뚱보조개 바다풍경점'] #위도,경도 못 불러오는 주소의 상호명 찾기

In [ ]:
sample['주소'][105]='제주특별자치도 제주시 동한두기길 39' #주소 찾아서 바꾸고 다시 위에 위도,경도 불러오기

In [ ]:
sample['위도'] = lat_list #불러온 위도와 경도를 sample에 추가 column으로 만들기
sample['경도'] = lng_list

In [ ]:
sample2 = df.rename(columns={'CMPNM_NM':'이름','ADSTRD_NM':'주소2','MLSFC_NM':'구분','SCLAS_NM':'구분2'})
sample2 #원 데이터의 컬럼명 바꾸기

In [ ]:
sample3 = pd.merge(sample,sample2,on='이름',how='inner') #원래의 데이터와 크롤링+위도경도 '이름'을 기준으로 데이터를 합침
sample3

In [39]:
sample3.to_excel('con_data/맛집_test2.xlsx',index = False) #데이터 저장

[{'이름': '화수목 제주'},
 {'이름': '바다119 제주'},
 {'이름': '신풍해장국 제주'},
 {'이름': '다다참치 제주'},
 {'이름': '로얄바 제주'},
 {'이름': '함경면옥 제주'}]

In [ ]:
import folium
import pandas as pd

In [ ]:
map=folium.Map(location=[sample3.위도.mean(), sample3.경도.mean()], zoom_start=10)
map

In [ ]:
tiles = "http://mt0.google.com/vt/lyrs=m&hl=ko&x={x}&y={y}&z={z}" #구글지도 타일 설정
attr= "Google" #속성 설정
map = folium.Map(location=[sample3['위도'].mean(), sample3['경도'].mean()], #folium을 사용하여 지도 초기화
                 tiles=tiles, attr=attr)
for i in sample3.index: #df의 각 행을 반복
    folium.Marker(
        location=[sample3.loc[i, '위도'], sample3.loc[i, '경도']],
        tooltip=sample3.loc[i, '이름'], #마커에 마우스를 올렸을 때 표시되는 툴팁으로 약국명 표시
        popup=folium.Popup(sample3.loc[i, '주소'], max_width=200),
        icon=folium.Icon(color='orange', icon='star')
    ).add_to(map)

In [ ]:
map